In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn import linear_model

from IPython.display import display

pd.options.display.max_columns = 50

In [4]:
df = pd.read_csv("../data/immo_data.csv")
desc = pd.read_csv("../data/immo_data_column_description.csv")

In [5]:
def drop_columns(df):
    """ Remove (supposedly) unimportant columns """
    return df.drop(
        [
            "scoutId",
            "houseNumber",
            "geo_bln",
            "geo_krs",
            "geo_plz",
            "date",
            "street",
            "streetPlain",
            "description",
            "facilities",
            "regio3",
            "firingTypes",
            "telekomHybridUploadSpeed",
            "totalRent",
            "baseRentRange",
        ],
        axis=1,
    )


def remove_outliers(df, lower_limit=0.005, upper_limit=0.995):
    """ Removing the (lower and upper) outliers """
    dfc = df.copy()
    columns_with_outliers = [
        "serviceCharge",
        "yearConstructed",
        "noParkSpaces",
        "baseRent",
        "livingSpace",
        "noRooms",
        "floor",
        "numberOfFloors",
        "heatingCosts",
        "lastRefurbish",
    ]
    
    # For each column we keep: Data that are < (99.5% quantile) and > (0.5% quantile) OR that are NaN (we will deal with this later). 
    upper_limits = df[columns_with_outliers].quantile(upper_limit)
    lower_limits = df[columns_with_outliers].quantile(lower_limit)
    
    for colname in columns_with_outliers:
        col = dfc[colname]
        dfc = dfc[
            ((col <= upper_limits[colname]) & (col >= lower_limits[colname]))
            | col.isna()
        ]
    return dfc


def remove_rows_with_NaN_target(df):
    """ Removing the records without a label """
    return df[df["baseRent"].isna() == False]


def impute_NaNs(df):
    """ Replacing NaNs with mean or most frequent """
    dfc = df.copy()
    categorical_columns = dfc.select_dtypes(exclude=np.number).columns
    imp_freq = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
    #dfc.loc[:, categorical_columns] = imp_freq.fit_transform(dfc[categorical_columns])
    dfc[categorical_columns] = imp_freq.fit_transform(dfc[categorical_columns])

    numeric_columns = dfc.select_dtypes(include=np.number).columns
    imp_mean = SimpleImputer(missing_values=np.nan, strategy="mean")
    #dfc.loc[:, numeric_columns] = imp_mean.fit_transform(dfc[numeric_columns])
    dfc[numeric_columns] = imp_mean.fit_transform(dfc[numeric_columns])
    return dfc


def print_evaluation(pipeline_or_model, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names, show_coeff=True):
    """ Output of R2 value, MSE and MAE for training and test set """
    r2_train = r2_score(y_train, y_train_pred)
    mse_train = mean_squared_error(y_train, y_train_pred)
    mae_train = mean_absolute_error(y_train, y_train_pred)

    r2_test = r2_score(y_test, y_test_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    
    print(
        f"{pipeline_or_model} Evaluation:\n"
        f"{'':6} {'R²':>10} | {'MSE':>14} | {'MAE':>10} | {'rows':>8} | {'columns':>8}\n"
        f"{'Train':6} {r2_train:10.5f} | {mse_train:14.2f} | {mae_train:10.2f} | {X_train.shape[0]:8} | {X_train.shape[1]:8}\n"
        f"{'Test':6} {r2_test:10.5f} | {mse_test:14.2f} | {mae_test:10.2f} | {X_test.shape[0]:8} | {X_test.shape[1]:8}\n"
    )
    
    # Output of the first 10 coefficients, scaled in descending order by absolute value
    coefficients_lr = pd.DataFrame({"Feature Name": feature_names, "Coefficient": pipeline_or_model.coef_})
    if show_coeff:
        display(coefficients_lr.sort_values("Coefficient", key=abs, ascending=False).head(10))
    
    # How many coefficients are non-zero?
    nonzero_coeff = sum(abs(coefficients_lr["Coefficient"])>1e-12)
    print(f"Number of nonzero coefficients: {nonzero_coeff}/{X_train.shape[1]}")

In [6]:
# Data pre-processing
df_reduced = drop_columns(df.sample(10000))
df_reduced = remove_outliers(df_reduced)
df_reduced = remove_rows_with_NaN_target(df_reduced)
df_reduced = impute_NaNs(df_reduced)
df_reduced = pd.get_dummies(df_reduced)
y = df_reduced.pop("baseRent")

# Training-Test-Split
X_train, X_test, y_train, y_test = train_test_split(df_reduced, y, test_size=0.2, random_state=0)

# Training
model_lr = linear_model.LinearRegression()
model_lr.fit(X_train, y_train)
y_train_pred = model_lr.predict(X_train)
y_test_pred = model_lr.predict(X_test)

# Evaluation
print_evaluation(model_lr, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names=df_reduced.columns)

LinearRegression() Evaluation:
               R² |            MSE |        MAE |     rows |  columns
Train     0.84971 |       23778.41 |     102.23 |     7728 |      510
Test   -65572754235221.42969 | 10914647454231087104.00 | 94160104.59 |     1932 |      510


,Feature Name,Coefficient
72,petsAllowed_no,-2.930620e+12
71,petsAllowed_negotiable,-2.930620e+12
73,petsAllowed_yes,-2.930620e+12
50,newlyConst_False,-1.346498e+12
51,newlyConst_True,-1.346498e+12
20,regio1_Berlin,-9.108420e+11
123,regio2_Berlin,6.556990e+11
55,hasKitchen_True,6.289675e+11
54,hasKitchen_False,6.289675e+11
225,regio2_Hamburg,-4.583336e+11


Number of nonzero coefficients: 509/510
